In [137]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [4]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [5]:
import os
from dotenv import load_dotenv
from openai import OpenAI
load_dotenv()


True

In [6]:
from typing import Tuple, List, Optional

In [7]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [8]:
from langchain_community.vectorstores import Neo4jVector

In [9]:
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key = OPENAI_API_KEY)

In [10]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.graphs import Neo4jGraph
from dotenv import load_dotenv

In [11]:
graph = Neo4jGraph()

/var/folders/qc/q94_1y514n3fvqnfz08sykkh0000gn/T/ipykernel_3319/267675202.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph()


In [12]:
# Load the paper
paper_loader = PyPDFLoader("../document/attention-is-all-you-need-Paper.pdf")
paper_documents = paper_loader.load()
print(f"Loaded {len(paper_documents)} pages from the paper")

Loaded 11 pages from the paper


In [16]:
# Load Wikipedia content about Transformer architecture
wiki_loader = WikipediaLoader(query="Attention Is All You Need")
wiki_documents = wiki_loader.load()
print(f"Loaded {len(wiki_documents)} sections from Wikipedia")

Loaded 25 sections from Wikipedia


In [13]:
# Combine documents
# all_documents = paper_documents  + wiki_documents
all_documents = paper_documents

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=500,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)
chunks = text_splitter.split_documents(all_documents)
print(f"Created {len(chunks)} chunks from all documents")

Created 11 chunks from all documents


In [15]:
# filtered_chunks = [Document(page_content=chunk.page_content) for chunk in chunks]

NameError: name 'Document' is not defined

In [16]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name="gpt-4o-mini")

In [17]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [19]:
graph_documents = llm_transformer.convert_to_graph_documents(chunks)

In [20]:
graph_documents

[GraphDocument(nodes=[Node(id='Ashish Vaswani', type='Person', properties={}), Node(id='Noam Shazeer', type='Person', properties={}), Node(id='Niki Parmar', type='Person', properties={}), Node(id='Jakob Uszkoreit', type='Person', properties={}), Node(id='Llion Jones', type='Person', properties={}), Node(id='Aidan N. Gomez', type='Person', properties={}), Node(id='Łukasz Kaiser', type='Person', properties={}), Node(id='Illia Polosukhin', type='Person', properties={}), Node(id='Google Brain', type='Organization', properties={}), Node(id='Google Research', type='Organization', properties={}), Node(id='University Of Toronto', type='Organization', properties={}), Node(id='Transformer', type='Concept', properties={}), Node(id='Wmt 2014 English-To-German Translation Task', type='Task', properties={}), Node(id='Wmt 2014 English-To-French Translation Task', type='Task', properties={})], relationships=[Relationship(source=Node(id='Ashish Vaswani', type='Person', properties={}), target=Node(id='G

In [21]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [22]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 500"

In [23]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [24]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [25]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [13]:
# delete_graph_step1 = "match (a) -[r] -> () delete a, r"
# delete_graph_step2 = "match (a) delete a"
# graph.query(delete_graph_step1)
# graph.query(delete_graph_step2)


[]

In [26]:
from typing import Tuple, List, Optional

In [27]:
from langchain_community.vectorstores import Neo4jVector

In [28]:
from langchain_openai import OpenAIEmbeddings
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [30]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [32]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the important entities that appear in the text",
    )


In [33]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [34]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting important entities from the text",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [35]:
entity_chain = prompt | llm.with_structured_output(Entities)

/Users/uddeshyabarnwal/miniconda3/lib/python3.12/site-packages/langchain_openai/chat_models/base.py:1624: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [36]:
entity_chain.invoke({"question": "what is architecture of of attention is all you need"})

Entities(names=['architecture', 'attention is all you need'])

In [37]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [38]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [39]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [41]:
print(structured_retriever("what is architecture of of attention is all you need"))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

Denny Britz - AUTHORED -> Massive Exploration Of Neural Machine Translation Architectures
Anna Goldie - AUTHORED -> Massive Exploration Of Neural Machine Translation Architectures
Minh-Thang Luong - AUTHORED -> Massive Exploration Of Neural Machine Translation Architectures
Quoc V . Le - AUTHORED -> Massive Exploration Of Neural Machine Translation Architectures


In [42]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [43]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [44]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [45]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [46]:
from langchain_core.output_parsers import StrOutputParser
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [47]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [48]:
prompt = ChatPromptTemplate.from_template(template)


In [49]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [50]:
chain.invoke({"question": "what is architecture of of attention is all you need"})

Search query: what is architecture of of attention is all you need


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

'The architecture of "Attention Is All You Need" introduces the Transformer model, which relies entirely on attention mechanisms without using recurrence or convolutions. It consists of an encoder-decoder structure, where the encoder maps input sequences to continuous representations using multi-head self-attention, and the decoder generates output sequences based on these representations. Both the encoder and decoder are composed of stacked layers that include residual connections and layer normalization. The model allows for significant parallelization and achieves state-of-the-art performance in machine translation tasks.'

In [51]:
chain.invoke(
    {
        "question": "what is architecture of of attention is all you need",
        "chat_history": [("attention is all you need  ", "Transformer")],
    }
)

Search query: What is the architecture of the attention is all you need model?


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL (node, node) { ... }} {position: line: 3, column: 13, offset: 104} for query: "CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})\n            YIELD node,score\n            CALL {\n              WITH node\n              MATCH (node)-[r:!MENTIONS]->(neighbor)\n              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output\n              UNION ALL\n              WITH node\n              MATCH (node)<-[r:!MENTIONS]-(neighbor)\n              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output\n            }\n            RETURN output LIMIT 50\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNoti

'The architecture of "Attention Is All You Need" is based on the Transformer model, which relies entirely on attention mechanisms without using recurrence or convolutions. It consists of an encoder-decoder structure, where the encoder maps input sequences to continuous representations, and the decoder generates output sequences. Both the encoder and decoder are composed of multiple identical layers that include multi-head self-attention and fully connected feed-forward networks. This design allows for significant parallelization and improved training efficiency.'